<a href="https://colab.research.google.com/github/KimGriezmann/Bigcontest_2021/blob/master/working_space_SeongGwan/split_coding_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import datetime


plt.rc('font', family='NanumBarunGothic')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def create_df():

  #연도별 데이터 경로
  HTS_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS'
  Player_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수'
  Hitter_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본'
  #단일 데이터 경로
  Team_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_팀.csv'
  Schedule_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_경기일정_2021.csv'
  val_dir = '/content/drive/MyDrive/빅콘테스트/02_평가데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_평가데이터.xlsx'

  year = [2018, 2019, 2020, 2021]

  for i in year:
    globals()['HTS_{}'.format(i)]     = pd.read_csv(HTS_dir + '_' + str(i) + '.csv', encoding='CP949')
    globals()['Player_{}'.format(i)]  = pd.read_csv(Player_dir + '_' + str(i) + '.csv', encoding='CP949')
    globals()['Hitter_{}'.format(i)]  = pd.read_csv(Hitter_dir + '_' + str(i) + '.csv', encoding='CP949')

  global Team_df, Schedule_df, val_df
  Team_df = pd.read_csv(Team_dir, encoding='CP949')
  Schedule_df = pd.read_csv(Schedule_dir, encoding='CP949')
  val_df = pd.read_excel(val_dir)

######################

#연도별 데이터 하나로 통합
def concat_df():
  df_list = ['HTS', 'Player', 'Hitter']

#각 칼럼이 모두 같은지 검사 후 통합
  for i in df_list:
    if ((globals()['{}_2018'.format(i)].columns == globals()['{}_2019'.format(i)].columns).all()) and ((globals()['{}_2019'.format(i)].columns == globals()['{}_2020'.format(i)].columns).all()) and \
    ((globals()['{}_2020'.format(i)].columns == globals()['{}_2021'.format(i)].columns).all()):
      globals()['{}_df'.format(i)] = pd.concat( [ globals()['{}_2018'.format(i)], globals()['{}_2019'.format(i)], globals()['{}_2020'.format(i)], globals()['{}_2021'.format(i) ] ] )

#연도별 길이의 합이 통합된 데이터의 길이와 같은지 검사
  for i in df_list:
    if len(globals()['{}_2018'.format(i)]) + len(globals()['{}_2019'.format(i)]) + len(globals()['{}_2020'.format(i)]) + len(globals()['{}_2021'.format(i)]) == len(globals()['{}_df'.format(i)]):
      print('{} is complete!'.format(i))

######################

create_df()
concat_df()

HTS is complete!
Player is complete!
Hitter is complete!


In [151]:
df_2018 = pd.read_csv('/content/drive/MyDrive/빅콘테스트/add_hts_2018.csv', encoding='CP949')
df_2019 = pd.read_csv('/content/drive/MyDrive/빅콘테스트/add_hts_2019.csv', encoding='CP949')
df_2020 = pd.read_csv('/content/drive/MyDrive/빅콘테스트/add_hts_2020.csv', encoding='CP949')
df_2021 = pd.read_csv('/content/drive/MyDrive/빅콘테스트/add_hts_2021.csv', encoding='CP949')

In [186]:
# 분할 함수
def order(df, period, interval, del_cri, except_print):

    first_date = df.sort_values('DATE').reset_index(drop=True).loc[0,'DATE']

    last_date = df.sort_values('DATE').reset_index(drop=True).loc[len(df)-1, 'DATE']

    # 결과값 저장할 데이터프레임 생성
    home_split = pd.DataFrame()
    away_split = pd.DataFrame()

    while True:
        
        start_date = str(first_date)

        end_date = datetime.datetime.strptime(start_date, '%Y%m%d')
        end_date = end_date + datetime.timedelta(days=period)
        end_date = end_date.strftime('%Y%m%d')

        pred_date = datetime.datetime.strptime(end_date, '%Y%m%d')
        pred_date = pred_date + datetime.timedelta(days=21)
        pred_date = pred_date.strftime('%Y%m%d')

        if int(pred_date) >= last_date:
            print('가져오는 날짜{}가 마지막 날짜{} 이후의 데이터입니다.'.format(pred_date, last_date))
            break

        # 해당 시즌 타자 목록 (name -> pcode)
        hitter_list = list(set(df['PCODE']))

        # 더해줄 칼럼 목록 (avg, obp, slg, ops, avLI, RE24 제외)
        target_col = ['ab', 'score', 'hit', '2_hit', '3_hit', 'homerun', 'tb', 'rbi', 'sb', 'cs', 'bb', 'hpp', 'ibb', 'so', 'gdp', 'sh', 'sf', 'pitch', 'WPA', 'power_barrel', 'contact_barrel', 'clutch_barrel']

        # start_date 이후의 경기만 추출
        tr_df = df[ (df['DATE'] >= int(start_date)) & (df['DATE'] <= int(end_date)) ]
        ts_df = df[ (df['DATE'] > int(end_date)) & (df['DATE'] <= int(pred_date)) ]

        # first_date + interval
        first_date = datetime.datetime.strptime(start_date, '%Y%m%d')
        first_date = first_date + datetime.timedelta(days=interval)
        first_date = first_date.strftime('%Y%m%d')

        # 타자 한명씩 추출
        for player in hitter_list:

            tr_pl_data = tr_df[tr_df['PCODE'] == player]
            ts_pl_data = ts_df[ts_df['PCODE'] == player]

            away_tr = tr_pl_data[ tr_pl_data['vs'].str.contains('@') ]
            home_tr = tr_pl_data[ tr_pl_data['vs'].str.contains('@')  == False ]

            away_ts = ts_pl_data[ ts_pl_data['vs'].str.contains('@') ]
            home_ts = ts_pl_data[ ts_pl_data['vs'].str.contains('@')  == False ]

            if (len(away_tr) < del_cri) | (len(home_tr) < del_cri) | (len(away_ts) < 3.5) | (len(home_ts) < 3.5) :
                if except_print == True:
                    print('{}은/는 경기 수가 {}개보다 작기 때문에 삭제'.format(player, del_cri))
                continue

            else:

                # tr_data의 타겟 칼럼을 모두 합해서 저장
                home_sum = home_tr.loc[:, target_col].sum()
                away_sum = away_tr.loc[:, target_col].sum()
                # 어떤 선수인지 저장 (name -> pcode)
                home_sum['PCODE'] = player
                away_sum['PCODE'] = player

                home_sum['NAME'] = tr_pl_data.iloc[0,0]
                away_sum['NAME'] = tr_pl_data.iloc[0,0]

                # ts_data의 성적들로 avg, slg, obp, ops 라벨 생성
                home_ts_sum = home_ts.loc[:, target_col].sum()
                away_ts_sum = away_ts.loc[:, target_col].sum()

                home_sum['avg_label'] = home_ts_sum['hit'] / home_ts_sum['ab'] if home_ts_sum['ab'] != 0 else 0
                home_sum['slg_label'] = home_ts_sum['tb'] / home_ts_sum['ab'] if home_ts_sum['ab'] != 0 else 0
                home_sum['obp_label'] = (home_ts_sum['hit'] + home_ts_sum['bb'] + home_ts_sum['hpp']) / (home_ts_sum['ab'] +  home_ts_sum['bb'] + home_ts_sum['hpp'] + home_ts_sum['sf'])
                home_sum['ops_label'] = home_sum['slg_label'] + home_sum['obp_label']

                away_sum['avg_label'] = away_ts_sum['hit'] / away_ts_sum['ab'] if away_ts_sum['ab'] != 0 else 0
                away_sum['slg_label'] = away_ts_sum['tb'] / away_ts_sum['ab'] if away_ts_sum['ab'] != 0 else 0
                away_sum['obp_label'] = (away_ts_sum['hit'] + away_ts_sum['bb'] + away_ts_sum['hpp']) / (away_ts_sum['ab'] +  away_ts_sum['bb'] + away_ts_sum['hpp'] + away_ts_sum['sf'])
                away_sum['ops_label'] = away_sum['slg_label'] + away_sum['obp_label']

                # Series 데이터를 DataFrame 형식으로 변환
                home_result = home_sum.to_frame().transpose()
                away_result = away_sum.to_frame().transpose()
                    
                # 이 선수 데이터를 전체 데이터셋에 추가
                home_split = pd.concat([home_split, home_result])
                away_split = pd.concat([away_split, away_result])
            
    # 데이터셋 인덱스 초기화
    home_split = home_split.reset_index(drop=True)
    away_split = away_split.reset_index(drop=True)
            
    return home_split, away_split

In [187]:
home_2018, away_2018 = order(df = df_2018, period = 100, interval = 7, del_cri= 10, except_print = False)

가져오는 날짜20181015가 마지막 날짜20181014 이후의 데이터입니다.


In [188]:
home_2018

,ab,score,hit,2_hit,3_hit,homerun,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,sf,pitch,WPA,power_barrel,contact_barrel,clutch_barrel,PCODE,NAME,avg_label,slg_label,obp_label,ops_label
0,138,36,51,14,2,8,93,42,1,1,22,1,5,22,2,0,8,638,1.287,23,10,7,76290,김현수,0.4,0.48,0.53125,1.01125
1,97,11,35,6,0,0,41,14,1,0,7,3,0,7,5,1,2,368,0.649,0,16,0,76802,김성현,0.266667,0.266667,0.47619,0.742857
2,91,16,22,3,3,1,34,8,2,3,6,0,1,13,0,0,0,368,0.655,3,8,0,64006,심우준,0.230769,0.230769,0.285714,0.516484
3,81,19,33,5,0,7,59,18,0,0,15,2,0,14,1,1,2,452,2.438,11,12,3,76812,이재원,0.352941,0.764706,0.47619,1.2409
4,155,23,48,14,2,6,84,27,4,3,10,1,0,38,2,0,0,635,0.409,12,10,2,76313,황재균,0.285714,0.571429,0.310345,0.881773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,96,13,27,7,0,2,40,16,3,2,12,3,0,10,1,0,0,439,-0.381,3,12,0,61404,김헌곤,0.173913,0.217391,0.24,0.457391
674,66,9,21,3,0,0,24,7,1,0,7,0,0,8,1,0,2,280,0.343,1,9,0,99810,이진영,0.380952,0.428571,0.380952,0.809524
675,112,20,34,9,0,5,58,26,6,0,13,3,2,24,2,0,3,494,1.246,9,9,4,62947,나성범,0.304348,0.521739,0.37037,0.89211
676,100,15,22,4,0,2,32,13,2,0,9,0,0,18,2,2,1,422,0.005,4,8,0,74215,윤석민,0.333333,0.52381,0.363636,0.887446


In [169]:
away_2018

,ab,score,hit,2_hit,3_hit,homerun,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,sf,pitch,WPA,power_barrel,contact_barrel,clutch_barrel,PCODE,avg_label,slg_label,obp_label,ops_label
0,148.0,28.0,52.0,14.0,0.0,6.0,84.0,28.0,0.0,0.0,8.0,0.0,1.0,20.0,2.0,0.0,1.0,555.0,1.488,15.0,20.0,5.0,76290.0,0.375000,0.625000,0.482759,1.107759
1,117.0,12.0,28.0,6.0,0.0,1.0,37.0,16.0,0.0,1.0,6.0,2.0,0.0,12.0,7.0,1.0,1.0,434.0,-0.536,0.0,16.0,0.0,76802.0,0.214286,0.285714,0.290323,0.576037
2,91.0,7.0,25.0,6.0,0.0,2.0,37.0,7.0,4.0,6.0,2.0,2.0,0.0,16.0,2.0,3.0,0.0,366.0,-0.527,1.0,11.0,0.0,64006.0,0.333333,0.466667,0.312500,0.779167
3,120.0,15.0,36.0,6.0,0.0,2.0,48.0,10.0,0.0,0.0,10.0,3.0,0.0,26.0,4.0,2.0,1.0,560.0,-0.605,8.0,17.0,0.0,76812.0,0.206897,0.275862,0.258065,0.533927
4,138.0,15.0,38.0,12.0,1.0,3.0,61.0,23.0,5.0,1.0,9.0,1.0,1.0,30.0,0.0,0.0,2.0,579.0,0.562,12.0,13.0,3.0,76313.0,0.352941,0.588235,0.476190,1.064426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,119.0,25.0,41.0,7.0,0.0,3.0,57.0,18.0,6.0,1.0,12.0,3.0,0.0,21.0,5.0,2.0,2.0,535.0,-0.266,8.0,16.0,0.0,61404.0,0.076923,0.076923,0.200000,0.276923
674,61.0,8.0,17.0,4.0,0.0,1.0,24.0,8.0,0.0,0.0,4.0,0.0,0.0,7.0,0.0,0.0,1.0,251.0,0.438,3.0,7.0,0.0,99810.0,0.263158,0.421053,0.333333,0.754386
675,125.0,26.0,39.0,9.0,2.0,4.0,64.0,17.0,5.0,1.0,13.0,2.0,1.0,24.0,3.0,0.0,0.0,540.0,0.679,13.0,11.0,3.0,62947.0,0.305556,0.472222,0.358974,0.831197
676,89.0,7.0,26.0,2.0,0.0,4.0,40.0,10.0,0.0,0.0,8.0,0.0,0.0,15.0,4.0,0.0,0.0,390.0,0.362,7.0,10.0,2.0,74215.0,0.250000,0.291667,0.280000,0.571667


# 휴지통

In [80]:
# 연속적 분할 함수
def order(df, period, interval, del_cri, except_print):

    first_date = df.sort_values('day').reset_index(drop=True).loc[0,'day']
    first_date = '0'+str(first_date)

    last_date = df.sort_values('day').reset_index(drop=True).loc[len(df)-1, 'day']
    last_date = '0'+str(last_date)

    while True:
        
        start_date = first_date

        end_date = datetime.datetime.strptime(start_date, '%m%d')
        end_date = end_date + datetime.timedelta(days=period)
        end_date = end_date.strftime('%m%d')

    pred_date = datetime.datetime.strptime(end_date, '%m%d')
    pred_date = pred_date + datetime.timedelta(days=21)
    pred_date = pred_date.strftime('%m%d')

    # 해당 시즌 타자 목록 (name -> pcode)
    hitter_list = list(set(df['PCODE']))

    # 더해줄 칼럼 목록 (avg, obp, slg, ops, avLI, RE24 제외)
    target_col = ['ab', 'score', 'hit', '2_hit', '3_hit', 'homerun', 'tb', 'rbi', 'sb', 'cs', 'bb', 'hpp', 'ibb', 'so', 'gdp', 'sh', 'sf', 'pitch', 'WPA']

    # start_date 이후의 경기만 추출
    tr_df = df[ (df['day'] >= int(start_date)) & (df['day'] <= int(end_date)) ]
    ts_df = df[ (df['day'] > int(end_date)) & (df['day'] <= int(pred_date)) ]

    # 결과값 저장할 데이터프레임 생성
    home_split = pd.DataFrame()
    away_split = pd.DataFrame()
        
    # 타자 한명씩 추출
    for player in hitter_list:

        tr_pl_data = tr_df[tr_df['PCODE'] == player]
        ts_pl_data = ts_df[ts_df['PCODE'] == player]

        if (len(tr_pl_data) < del_cri) | (len(ts_pl_data) < del_cri) :
            if except_print == True:
                print('{}은/는 경기 수가 {}개보다 작기 때문에 삭제'.format(player, del_cri))
            continue

        else:
            away_tr = tr_pl_data[ tr_pl_data['vs'].str.contains('@') ]
            home_tr = tr_pl_data[ tr_pl_data['vs'].str.contains('@')  == False ]

            away_ts = ts_pl_data[ ts_pl_data['vs'].str.contains('@') ]
            home_ts = ts_pl_data[ ts_pl_data['vs'].str.contains('@')  == False ]
                
            # tr_data의 타겟 칼럼을 모두 합해서 저장
            home_sum = home_tr.loc[:, target_col].sum()
            away_sum = away_tr.loc[:, target_col].sum()
            # 어떤 선수인지 저장 (name -> pcode)
            home_sum['PCODE'] = player
            away_sum['PCODE'] = player

            # ts_data의 성적들로 avg, slg, obp, ops 라벨 생성
            home_ts_sum = home_ts.loc[:, target_col].sum()
            away_ts_sum = away_ts.loc[:, target_col].sum()

            home_sum['avg_label'] = home_ts_sum['hit'] / home_ts_sum['ab']
            home_sum['slg_label'] = home_ts_sum['tb'] / home_ts_sum['ab']
            home_sum['obp_label'] = (home_ts_sum['hit'] + home_ts_sum['bb'] + home_ts_sum['hpp']) / (home_ts_sum['ab'] +  home_ts_sum['bb'] + home_ts_sum['hpp'] + home_ts_sum['sf'])
            home_sum['ops_label'] = home_sum['slg_label'] + home_sum['obp_label']

            away_sum['avg_label'] = away_ts_sum['hit'] / away_ts_sum['ab']
            away_sum['slg_label'] = away_ts_sum['tb'] / away_ts_sum['ab']
            away_sum['obp_label'] = (away_ts_sum['hit'] + away_ts_sum['bb'] + away_ts_sum['hpp']) / (away_ts_sum['ab'] +  away_ts_sum['bb'] + away_ts_sum['hpp'] + away_ts_sum['sf'])
            away_sum['ops_label'] = away_sum['slg_label'] + away_sum['obp_label']

            # Series 데이터를 DataFrame 형식으로 변환
            home_result = home_sum.to_frame().transpose()
            away_result = away_sum.to_frame().transpose()
                
            # 이 선수 데이터를 전체 데이터셋에 추가
            home_split = pd.concat([home_split, home_result])
            away_split = pd.concat([away_split, away_result])
        
    # 데이터셋 인덱스 초기화
    home_split = home_split.reset_index(drop=True)
    away_split = away_split.reset_index(drop=True)
        
    return home_split, away_split